In [21]:
%%timeit

import aocd

data = [ (v[0], int(v[1:])) for v in aocd.get_data(day=1, year=2025).splitlines() ]
part1, part2, cur, pd = 0, 0, 50, "R"

for d,v in data:
    if d != pd: pd, cur = d, (100-cur)%100
    md, cur = divmod(cur+v,100)
    part1, part2 = part1 + (cur == 0), part2 + md
    
#print("Part1:",part1,"Part2:",part2)

3.19 ms ± 648 μs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [22]:
%%timeit

import aocd

data = [ v.split("-") for v in aocd.get_data(day=2, year=2025).split(",") ]
part1, part2 = 0, 0

def bad(fro, to, rep):
    min_l = (len(fro)-1)//rep+1
    max_l = len(to)//rep
    if min_l>max_l: return set()
    mi = (10**(min_l-1)) if len(fro)%rep else (int(fro[:min_l]) + (fro[:min_l]*(rep-1)<fro[min_l:]))
    ma = (10**(max_l)-1) if len(to)%rep else (int(to[:max_l]) - (to[:max_l]*(rep-1)>to[max_l:]))
    return {int(str(v)*rep) for v in range(mi,ma+1)}

for fro,to in data:
    s = set(bad(fro,to,2))
    part1 += sum(s)
    for k in range(3,len(to)+1): s.update(bad(fro,to,k))
    part2 += sum(s)

#print("Part1:",part1,"Part2:",part2)

872 μs ± 139 μs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [23]:
%%timeit

import aocd

data = [ v.split("-") for v in aocd.get_data(day=2, year=2025).split(",") ]
part1, part2 = 0, 0

# 2 3 5 -6 -10 

def bad(fro, to, rep):
    ret = 0
    min_l = (len(fro)-1)//rep+1
    max_l = len(to)//rep
    if min_l>max_l: return ret
    mi = (10**(min_l-1)) if len(fro)%rep else (int(fro[:min_l]) + (fro[:min_l]*(rep-1)<fro[min_l:]))
    ma = (10**(max_l)-1) if len(to)%rep else (int(to[:max_l]) - (to[:max_l]*(rep-1)>to[max_l:]))
    mil = len(str(mi))
    mal = len(str(ma))
    for l in range(mil,mal+1):
        if l == mil: tmi = mi
        else: tmi = 10*(mil-1)
        if l == mal: tma = ma
        else: tma = 10*(mil) - 1
        ret += (tmi+tma)*(tma-tmi+1)//2*sum((10**(k*l)) for k in range(rep))
    return ret

for fro,to in data:
    s = bad(fro,to,2)
    part1 += s
    part2 += s
    for k in [3,5,7,-6,-10]: part2 += k//abs(k)*bad(fro,to,abs(k))

#print("Part1:",part1,"Part2:",part2)



634 μs ± 54.1 μs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [25]:
%%timeit

import aocd

def getjolt(batt,k):
    res, ret, cur, n = batt[-k:], 0, -1, len(batt)
    for i in range(len(batt)):
        while cur>=0 and res[cur]<batt[i] and n-i>=k-cur: cur -= 1
        if cur<k-1: res[cur+1], cur = batt[i], cur+1
    for v in res: ret = ret*10+v
    return ret

input = aocd.get_data(day=3, year=2025)
data = [[int(char) for char in line] for line in input.splitlines()]

#print
("Part1:",sum(map(lambda x:getjolt(x,2),data)),"Part1:",sum(map(lambda x:getjolt(x,12),data)))

8.39 ms ± 352 μs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [26]:
%%timeit

import aocd
import numpy as np
input = aocd.get_data(day=4, year=2025)

mt = np.matrix([[c=="@" for c in line] for line in input.splitlines()])
h, w = mt.shape

def removable_rolls(mt):
    t = np.zeros((h,w))
    t[:h-1,:w-1] += mt[1:,1:]
    t[1:,1:] += mt[:h-1,:w-1]

    t[:h-1,:] += mt[1:,:]
    t[1:,:] += mt[:h-1,:]

    t[:,1:] += mt[:,:w-1]
    t[:,:w-1] += mt[:,1:]

    t[:h-1,1:] += mt[1:,:w-1]
    t[1:,:w-1] += mt[:h-1,1:]
    return np.logical_and((t<4),mt)

rm = removable_rolls(mt)
rm_size = np.matrix.sum(rm)
part1 = part2 = rm_size
while rm_size:
    mt = np.logical_xor(rm, mt)
    rm = removable_rolls(mt)
    rm_size = np.matrix.sum(rm)
    part2 += rm_size

#print("Part1:",part1,"Part2:",part2)



20.5 ms ± 1.96 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [27]:
%%timeit

import aocd
import numpy as np
input = aocd.get_data(day=4, year=2025)

mat = {(i,j) for i,line in enumerate(input.splitlines()) for j,c in enumerate(line) if c == "@" }

def neb(i,j): return({(i-1,j-1),(i-1,j+1),(i-1,j),(i,j-1),(i,j+1),(i+1,j-1),(i+1,j),(i+1,j+1)})

acc = { p for p in mat if len(neb(*p) & mat)<4 }
part1 = part2 = len(acc)
while len(acc):
    mat -= acc
    acc = { p for p in mat if len(neb(*p) & mat)<4 }
    part2 += len(acc)
    
#print("Part1:",part1,"Part2:",part2)


512 ms ± 26.4 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [33]:
%%timeit

import aocd

input = aocd.get_data(day=5, year=2025)
import bisect

R,L = input.split("\n\n")
R = sorted((tuple(map(int,r.split("-"))) for r in R.splitlines()))
L = list(map(int,L.splitlines()))

R2 = []
cur = -1
for a,b in R:
    if a <= cur:
        if b>=cur:
            R2[-1] = (R2[-1][0],b)
            cur = b+1
    elif a > cur:
        R2.append((a,b))
        cur = b+1
        
part1 = sum(v>=R2[p][0] for v in L for p in [bisect.bisect_left(R2,v,key=lambda x:x[1])] if p<len(R2))
part2 = sum(b-a+1 for a,b in R2)
#print("Part1:",part1,"Part2:",part2)


1.8 ms ± 218 μs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [29]:
%%timeit

import aocd
from functools import reduce

input = aocd.get_data(day=6, year=2025)

data = input.splitlines()
nums = [list(map(int,line.split())) for line in data[:-1]]
signs = data[-1].split()

part1 = sum(reduce((lambda x,y:x+y) if signs[i]=="+" else (lambda x,y:x*y),
                   (nums[j][i] for j in range(len(nums)))) for i in range(len(signs)))

part2 = 0
op = (lambda x,y:x+y)
bloc = 0
for j in range(len(data[0])):
    cur = 0
    for i in range(len(data)-1):
        c = data[i][j]
        if c !=" ": cur = cur*10+(ord(c)-48)

    if cur == 0:
        part2 += bloc
        bloc = 0
        op = (lambda x,y:x+y)

    else: bloc = op(bloc,cur)

    if data[-1][j]!=" ":
        op = (lambda x,y:x+y) if data[-1][j]=="+" else (lambda x,y:x*y)
part2 += bloc

#print("Part1:",part1,"Part2:",part2)





6.56 ms ± 666 μs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [30]:
%%timeit

import aocd
from functools import reduce

input = aocd.get_data(day=6, year=2025)

data = input.splitlines()
nums = [list(map(int,line.split())) for line in data[:-1]]
signs = data[-1].split()

part1 = sum(reduce((lambda x,y:x+y) if signs[i]=="+" else (lambda x,y:x*y),
                   (nums[j][i] for j in range(len(nums)))) for i in range(len(signs)))
                   
data2 = ["".join(data[i][j] for i in range(len(data)-1)).split() for j in range(len(data[0])) ]

part2 = 0
op = (lambda x,y:x+y)
bloc = 0
for i,l in enumerate(data2):
    if len(l):
        bloc = op(int(l[0]),bloc)
        if data[-1][i] != " ": op = (lambda x,y:x+y) if data[-1][i]=="+" else (lambda x,y:x*y)
    else: op, bloc, part2 = (lambda x,y:x+y), 0, part2+bloc
part2 += bloc

#print("Part1:",part1,"Part2:",part2)




10.8 ms ± 618 μs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [130]:
%%timeit
import aocd
from collections import Counter

input = aocd.get_data(day=7, year=2025)
data = input.splitlines()
beams = Counter({i:1 for i,v in enumerate(data[0]) if v == "S"})
part1 = 0

for line in data[2::2]:
    keys = [k for k in beams.keys()]
    for i in keys:
        if line[i] == "^":
            part1 += 1
            p = beams.pop(i)
            beams[i+1] += p
            beams[i-1] += p
part2 = sum(beams.values())

#print("Part1:",part1,"Part2:",part2)

1.63 ms ± 65.8 μs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [132]:
%%timeit
import aocd
import numpy as np

input = aocd.get_data(day=7, year=2025)
data = input.splitlines()
cur = np.array([1*(v == "S") for i,v in enumerate(data[0])], dtype=np.int64)
part1 = 0

for line in data[2::2]:
    la = np.array([c=="^" for c in line])
    part1 += sum((cur>0)*la)
    nc = cur*(1-la)
    nc[:-1] += la[1:]*cur[1:]
    nc[1:] += la[:-1]*cur[:-1]
    cur = nc
part2 = sum(cur)

#print("Part1:",part1,"Part2:",part2)

2.48 ms ± 61.3 μs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [131]:
%%timeit
import aocd
from collections import Counter
import heapq

input = aocd.get_data(day=7, year=2025)
data = input.splitlines()
n,m = len(data)//2,len(data[0])
jumps = [0]*(n+1)
jumps[n] = [n]*m

s = next(i for i,v in enumerate(data[0]) if v == "S")
for i in range(n-1,-1,-1):
    jumps[i] = [i if data[2*i][j] == "^" else jumps[i+1][j] for j in range(m)]

heap = [(jumps[0][s],s,1)]
part1 = part2 = 0

while len(heap):
    l,v,c = heapq.heappop(heap)
    
    while len(heap) and heap[0][:2] == (l,v):
        _,_,c2 = heapq.heappop(heap)
        c += c2
    if l == n: part2 += c
    else:
        heapq.heappush(heap,(jumps[l][v-1],v-1,c))
        heapq.heappush(heap,(jumps[l][v+1],v+1,c))
        part1 += 1
    
#print("Part1:",part1,"Part2:",part2)



3.36 ms ± 253 μs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [162]:
#%%timeit

from itertools import combinations
import heapq
import aocd
from collections import defaultdict

input, pairs = aocd.get_data(day=8, year=2025), 1000
data = [tuple(map(int,line.split(","))) for line in input.splitlines()]
heap = []
part2di = {}

def sqdiff(a,b): return sum((x-y)**2 for x,y in zip(a,b))

for a,b in combinations(data,2):
    dif = sqdiff(a,b)
    if a in part2di: part2di[a] = min(part2di[a], (sqdiff(a,b), a[0]*b[0]))
    else: part2di[a] = (sqdiff(a,b), a[0]*b[0])
    if b in part2di: part2di[b] = min(part2di[b], (sqdiff(a,b), a[0]*b[0]))
    else: part2di[b] = (sqdiff(a,b), a[0]*b[0])
    heapq.heappush(heap,(-sqdiff(a,b), a, b))
    while len(heap)>pairs: heapq.heappop(heap)

di = defaultdict(list)

for _,a,b in heap:
    di[a].append(b)
    di[b].append(a)

#print(di)

s = set(di.keys())
sizes = []

while len(s):
    key = s.pop()
    gsize = 0
    gp = set([key])
    while len(gp):
        gsize += len(gp)
        gp = set(k2 for k1 in gp for k2 in di[k1] if k2 in s)
        s -= gp
    sizes.append(gsize)

sizes.sort(reverse=True)
part1 = sizes[0]*sizes[1]*sizes[2]
part2 = max(part2di.values())[1]

print("Part1:",part1,"Part2:",part2)


Part1: 122430 Part2: 8135565324


In [206]:
%%timeit

import numpy as np
import aocd

class UnionFind:
    def __init__(self, n):
        self.parent = list(range(n))
        self.rank = [0] * n
        self.size = [1] * n
        self.count = n

    def find(self, x):
        while self.parent[x] != x:
            self.parent[x] = self.parent[self.parent[x]]
            x = self.parent[x]
        return x

    def union(self, a, b):
        ra = self.find(a)
        rb = self.find(b)
        if ra == rb: return False
        if self.rank[ra] < self.rank[rb]: ra, rb = rb, ra
        self.parent[rb] = ra
        self.size[ra] += self.size[rb]
        if self.rank[ra] == self.rank[rb]: self.rank[ra] += 1
        self.count -= 1
        return True

    def component_sizes(self):
        return [self.size[i] for i in range(len(self.parent)) if self.parent[i] == i]

    def largest_k(self, k=3):
        return sorted(self.component_sizes(), reverse=True)[:k]

input = aocd.get_data(day=8, year=2025)
data = [tuple(map(int,line.split(","))) for line in input.splitlines()]

X = np.asarray(data, dtype=np.int64)

G = X @ X.T
sq = np.sum(X**2, axis=1)
dist2 = sq[:, None] + sq[None, :] - 2*G
i, j = np.triu_indices(len(X), k=1)
order = np.argsort(dist2[i, j], kind="quicksort")

uf = UnionFind(1000)
for k, (ii, jj) in enumerate(np.column_stack((i[order], j[order]))):
    if uf.union(ii, jj):
        if uf.count == 1:
            part2 = (data[ii][0]*data[jj][0])
            break
    if k == 999:
        a,b,c = uf.largest_k()
        part1 = a*b*c

#print("Part1:",part1,"Part2:",part2)

67.3 ms ± 3.02 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [281]:
%%timeit
from itertools import combinations

input = aocd.get_data(day=9, year=2025)
data = [tuple(map(int,line.split(","))) for line in input.splitlines()]
n = len(data)

part1 = max( (abs(a[0]-b[0])+1)*(abs(a[1]-b[1])+1) for a,b in combinations(data,2))
part2 = 0

segments = sorted([ (data[i][1],min(data[i][0],data[(i+1)%n][0]),max(data[i][0],data[(i+1)%n][0])) for i in range(n) if data[i][1] == data[(i+1)%n][1]])
lines_left, lines_right, points  = set(), set(), []

for x, miy, may in segments:
    for xp,yp,miyp,mayp in points:
        if miyp<=miy<=mayp: part2 = max(part2, (x-xp+1)*(abs(yp-miy)+1))
        if miyp<=may<=mayp: part2 = max(part2, (x-xp+1)*(abs(yp-may)+1))

    if miy in lines_left:
        if may in lines_right:
            lines_left.remove(miy)
            lines_right.remove(may)
            points = [ (xp,yp,miyp if yp<miy else max(may+1,miyp),mayp if yp>may else min(miy-1,mayp)) for xp,yp,miyp,mayp in points if yp<miy or yp>may ]

            continue
        else:
            lines_left.remove(miy)
            lines_left.add(may)
            npoints = [ (xp,yp,max(miyp,may),mayp) for xp,yp,miyp,mayp in points if may<=yp ]
            nmay = min(v for v in lines_right if v>may)
            npoints.append((x,may,may,nmay))
            points = npoints
            continue

    if may in lines_right:
            lines_right.remove(may)
            lines_right.add(miy)
            npoints = [ (xp,yp,miyp,min(mayp,miy)) for xp,yp,miyp,mayp in points if miy>=yp ]
            nmiy = max(v for v in lines_left if v<miy)
            npoints.append((x,miy,nmiy,miy))
            points = npoints
            continue
    points.append((x,miy,miy,may))
    points.append((x,may,miy,may))
    lines_left.add(miy)
    lines_right.add(may)

#print("Part1:",part1,"Part2:",part2)

29.1 ms ± 799 μs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [271]:
idx_x = {v:i for i,v in enumerate(sorted(set(x for x,y in data)))}
idx_y = {v:i for i,v in enumerate(sorted(set(y for x,y in data)))}

n = len(data)



arr = [["."]*len(idx_y) for _ in range(len(idx_x))]


for k in range(n):
    k2 = (k+1)%n
    i,j = data[k]
    i2,j2 = data[k2]

    arr[idx_x[i]][idx_y[j]] = "#"
    
    if i>i2: i,i2=i2,i
    if j>j2: j,j2=j2,j



    if i == i2:
        idxi = idx_x[i]
        for idxj in range(idx_y[j]+1,idx_y[j2]):
            arr[idxi][idxj] = "*"
    if j == j2:
        idxj = idx_y[j]
        for idxi in range(idx_x[i]+1,idx_x[i2]):
            arr[idxi][idxj] = "*"
    

print("\n".join("".join(c for c in line) for line in arr))


.....................................................................................................................#*#................................................................................................................................
.....................................................................................................................*.#*#..............................................................................................................................
.....................................................................................................................*...*.....#*#......................................................................................................................
.....................................................................................................................*...*...#*#.*......................................................................................................................
....

In [ ]:
def crossing(i,j):
    if (i+1)%n == j or (j+1)%n == i: return False
    i1 = (i+1)%n
    j1 = (j+1)%n
    if (data[i][0] == data[i1][0])^(data[j][0] == data[j1][0]):
        if data[i][0] == data[i1][0]: i,i1,j,j1 = j,j1,i,i1
        mii,mai = min(data[i][0],data[i1][0]),max(data[i][0],data[i1][0])
        mij,maj = min(data[j][1],data[j1][1]),max(data[j1][1],data[j][1])
        return (mii<=data[j][0]<=mai) and (mij<=data[i][0]<=maj)
    return False

for i in range(n-1):
    for j in range(i+2,n):
        if crossing(i,j):
            print(data[i],data[(i+1)%n],data[j],data[(j+1)%n],i,j)